In [2]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

class SimpleBinarySearchAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Binary Search Time Analyzer")
        self.root.geometry("1200x800")  # Made wider for better graph visibility
        
        # Store analysis results
        self.analysis_results = []
        
        # Setup GUI
        self.setup_gui()
        
    def recursive_binary_search(self, arr, target, left=0, right=None, steps=0):
        """Recursive binary search with step counting"""
        if right is None:
            right = len(arr) - 1
        
        steps += 1
        
        if left > right:
            return -1, steps
        
        mid = (left + right) // 2
        
        if arr[mid] == target:
            return mid, steps
        elif arr[mid] > target:
            return self.recursive_binary_search(arr, target, left, mid - 1, steps)
        else:
            return self.recursive_binary_search(arr, target, mid + 1, right, steps)
    
    def setup_gui(self):
        """Setup the main GUI"""
        # Create notebook for tabs
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Search Tab
        self.search_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.search_frame, text="Search")
        
        # Analysis Tab
        self.analysis_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.analysis_frame, text="Analysis")
        
        self.setup_search_tab()
        self.setup_analysis_tab()
    
    def setup_search_tab(self):
        """Setup the search interface"""
        # Title
        title = tk.Label(self.search_frame, text="Binary Search Time Analyzer", 
                        font=("Arial", 16, "bold"))
        title.pack(pady=10)
        
        # Input frame
        input_frame = tk.LabelFrame(self.search_frame, text="Input", font=("Arial", 12, "bold"))
        input_frame.pack(fill=tk.X, padx=20, pady=10)
        
        # Array size input
        size_frame = tk.Frame(input_frame)
        size_frame.pack(fill=tk.X, padx=10, pady=5)
        
        tk.Label(size_frame, text="Array Size:", font=("Arial", 11)).pack(side=tk.LEFT)
        self.size_var = tk.StringVar(value="1000")
        size_entry = tk.Entry(size_frame, textvariable=self.size_var, width=15)
        size_entry.pack(side=tk.LEFT, padx=10)
        
        tk.Label(size_frame, text="(Max: 1,000,000)", font=("Arial", 9), fg="gray").pack(side=tk.LEFT, padx=5)
        
        tk.Button(size_frame, text="Generate Array", command=self.generate_array,
                 bg="#4CAF50", fg="white", font=("Arial", 10)).pack(side=tk.LEFT, padx=10)
        
        # Array display
        array_frame = tk.LabelFrame(self.search_frame, text="Generated Array", 
                                   font=("Arial", 12, "bold"))
        array_frame.pack(fill=tk.X, padx=20, pady=10)
        
        self.array_text = scrolledtext.ScrolledText(array_frame, height=3, font=("Courier", 10))
        self.array_text.pack(fill=tk.X, padx=10, pady=5)
        
        # Target input
        target_frame = tk.Frame(self.search_frame)
        target_frame.pack(fill=tk.X, padx=20, pady=10)
        
        tk.Label(target_frame, text="Target Element:", font=("Arial", 11)).pack(side=tk.LEFT)
        self.target_var = tk.StringVar()
        tk.Entry(target_frame, textvariable=self.target_var, width=10).pack(side=tk.LEFT, padx=10)
        
        tk.Button(target_frame, text="Search", command=self.search_element,
                 bg="#2196F3", fg="white", font=("Arial", 10, "bold")).pack(side=tk.LEFT, padx=10)
        
        # Results display
        results_frame = tk.LabelFrame(self.search_frame, text="Search Results", 
                                     font=("Arial", 12, "bold"))
        results_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        self.results_text = scrolledtext.ScrolledText(results_frame, height=10, 
                                                     font=("Courier", 10))
        self.results_text.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Current array storage
        self.current_array = []
    
    def setup_analysis_tab(self):
        """Setup the analysis display tab"""
        # Create main container for better layout
        main_container = tk.Frame(self.analysis_frame)
        main_container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Analysis results table (top half)
        table_frame = tk.LabelFrame(main_container, text="Analysis History", 
                                   font=("Arial", 12, "bold"))
        table_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Create treeview
        columns = ('Array Size', 'Target', 'Found', 'Position', 'Steps', 'Time (μs)', 'Max Steps')
        self.analysis_tree = ttk.Treeview(table_frame, columns=columns, show='headings', height=6)
        
        for col in columns:
            self.analysis_tree.heading(col, text=col)
            self.analysis_tree.column(col, width=100, anchor=tk.CENTER)
        
        # Scrollbar for table
        table_scrollbar = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=self.analysis_tree.yview)
        self.analysis_tree.configure(yscrollcommand=table_scrollbar.set)
        
        # Pack table and scrollbar
        table_container = tk.Frame(table_frame)
        table_container.pack(fill=tk.X, padx=5, pady=5)
        
        self.analysis_tree.pack(side=tk.LEFT, fill=tk.X, expand=True)
        table_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Graph frame (bottom half)
        graph_frame = tk.LabelFrame(main_container, text="Performance Graph", 
                                   font=("Arial", 12, "bold"))
        graph_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create matplotlib figure with better sizing for single plot
        self.figure = Figure(figsize=(10, 6), dpi=100, facecolor='white')
        self.canvas = FigureCanvasTkAgg(self.figure, graph_frame)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Initialize empty graph
        self.initialize_empty_graph()
        
        # Control buttons
        button_frame = tk.Frame(graph_frame)
        button_frame.pack(pady=5)
        
        tk.Button(button_frame, text="Update Graph", command=self.update_graph,
                 bg="#FF9800", fg="white", font=("Arial", 10)).pack(side=tk.LEFT, padx=5)
        
        tk.Button(button_frame, text="Clear Analysis", command=self.clear_analysis,
                 bg="#F44336", fg="white", font=("Arial", 10)).pack(side=tk.LEFT, padx=5)
        
        tk.Button(button_frame, text="Auto Update", command=self.toggle_auto_update,
                 bg="#9C27B0", fg="white", font=("Arial", 10)).pack(side=tk.LEFT, padx=5)
        
        # Auto update toggle
        self.auto_update = False
    
    def initialize_empty_graph(self):
        """Initialize empty graph with proper layout"""
        self.figure.clear()
        
        # Create single plot for time vs array size
        ax = self.figure.add_subplot(1, 1, 1)
        
        # Configure empty plot
        ax.set_title('Binary Search: Execution Time vs Array Size', fontsize=14, fontweight='bold')
        ax.set_xlabel('Array Size (n)', fontsize=12)
        ax.set_ylabel('Execution Time (microseconds)', fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.text(0.5, 0.5, 'No data yet\nRun some searches to see performance results', 
                ha='center', va='center', transform=ax.transAxes, 
                fontsize=12, alpha=0.6, style='italic')
        
        self.figure.tight_layout()
        self.canvas.draw()
    
    def toggle_auto_update(self):
        """Toggle automatic graph updates"""
        self.auto_update = not self.auto_update
        status = "enabled" if self.auto_update else "disabled"
        messagebox.showinfo("Auto Update", f"Automatic graph updates {status}")
    
    def generate_array(self):
        """Generate and display a random sorted array"""
        try:
            size = int(self.size_var.get())
            if size <= 0 or size > 1000000:
                raise ValueError("Size must be between 1 and 1,000,000")
            
            # Generate random array and sort it (more efficient for large arrays)
            if size <= 10000:
                # For smaller arrays, use random sampling (shows actual values)
                self.current_array = sorted(random.sample(range(1, size * 3), size))
                array_display = f"Array: {self.current_array}"
            else:
                # For larger arrays, generate sequential array (more efficient)
                self.current_array = list(range(1, size * 2, 2))  # Odd numbers
                array_display = f"Array: [1, 3, 5, 7, ..., {self.current_array[-3]}, {self.current_array[-2]}, {self.current_array[-1]}] (size: {size})"
            
            # Display array (truncated for large arrays)
            self.array_text.delete('1.0', tk.END)
            if size <= 100:
                self.array_text.insert('1.0', array_display)
            else:
                # Show first and last few elements for large arrays
                first_10 = self.current_array[:10]
                last_10 = self.current_array[-10:]
                truncated_display = f"Array: {first_10} ... {last_10}\nSize: {size} elements\nRange: {self.current_array[0]} to {self.current_array[-1]}"
                self.array_text.insert('1.0', truncated_display)
            
            # Clear previous results
            self.results_text.delete('1.0', tk.END)
            self.results_text.insert('1.0', f"Generated sorted array of size {size:,}\n")
            if size <= 100:
                self.results_text.insert(tk.END, f"Array elements: {self.current_array}\n\n")
            else:
                self.results_text.insert(tk.END, f"Array range: {self.current_array[0]} to {self.current_array[-1]}\n")
                self.results_text.insert(tk.END, f"Sample elements: {self.current_array[:5]} ... {self.current_array[-5:]}\n\n")
            
        except ValueError as e:
            messagebox.showerror("Invalid Input", f"Please enter a valid array size (1 to 1,000,000)\nError: {e}")
    
    def search_element(self):
        """Search for target element and analyze performance"""
        if not self.current_array:
            messagebox.showwarning("No Array", "Please generate an array first")
            return
        
        try:
            target = int(self.target_var.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid target number")
            return
        
        array_size = len(self.current_array)
        # Measure search time with higher precision for large arrays
        num_trials = min(100, max(10, 1000000 // array_size))
        times = []
        
        for _ in range(num_trials):
            start_time = time.perf_counter()
            position, steps = self.recursive_binary_search(self.current_array, target)
            end_time = time.perf_counter()
            times.append((end_time - start_time) * 1_000_000)
        
        execution_time = sum(times) / len(times)  # Average time
        
        # Calculate theoretical maximum steps
        max_steps = int(np.ceil(np.log2(array_size)))
        
        # Display results
        self.results_text.delete('1.0', tk.END)
        
        result_text = f"SEARCH ANALYSIS\n"
        result_text += f"="*50 + "\n"
        result_text += f"Array Size: {array_size:,}\n"
        result_text += f"Target Element: {target:,}\n"
        result_text += f"Found: {'Yes' if position != -1 else 'No'}\n"
        
        if position != -1:
            result_text += f"Position: Index {position:,} (array[{position}] = {self.current_array[position]:,})\n"
        else:
            result_text += f"Position: Not found\n"
        
        result_text += f"Steps Taken: {steps}\n"
        result_text += f"Average Execution Time: {execution_time:.3f} microseconds ({num_trials} trials)\n"
        result_text += f"Theoretical Max Steps: {max_steps} (⌈log₂({array_size:,})⌉)\n"
        result_text += f"Efficiency: {steps}/{max_steps} = {(steps/max_steps)*100:.1f}% of maximum\n"
        
        # Time complexity analysis
        result_text += f"\nTIME COMPLEXITY ANALYSIS:\n"
        result_text += f"-"*30 + "\n"
        result_text += f"Algorithm: O(log n)\n"
        result_text += f"For n = {array_size:,}: log₂({array_size:,}) = {np.log2(array_size):.2f}\n"
        result_text += f"Actual steps ({steps}) vs Theoretical max ({max_steps})\n"
        
        # Performance comparison
        if array_size >= 1000:
            comparison_sizes = [1000, 10000, 100000, 1000000]
            result_text += f"\nSCALABILITY COMPARISON:\n"
            result_text += f"-"*25 + "\n"
            for comp_size in comparison_sizes:
                if comp_size <= array_size:
                    comp_steps = int(np.ceil(np.log2(comp_size)))
                    result_text += f"n = {comp_size:7,}: ≤ {comp_steps:2} steps\n"
        
        if steps <= max_steps:
            result_text += f"\n✓ Performance within expected O(log n) bounds\n"
        else:
            result_text += f"\n⚠ Steps exceeded theoretical maximum (unusual)\n"
        
        self.results_text.insert('1.0', result_text)
        
        # Store in analysis results
        analysis_entry = {
            'array_size': array_size,
            'target': target,
            'found': position != -1,
            'position': position if position != -1 else 'Not Found',
            'steps': steps,
            'time': execution_time,
            'max_steps': max_steps
        }
        
        self.analysis_results.append(analysis_entry)
        self.update_analysis_table()
        
        # Always update graph after each search
        self.update_graph()
        
        # Switch to analysis tab to show results
        self.notebook.select(1)
    
    def update_analysis_table(self):
        """Update the analysis results table"""
        # Clear existing items
        for item in self.analysis_tree.get_children():
            self.analysis_tree.delete(item)
        
        # Add all analysis results
        for result in self.analysis_results:
            values = (
                f"{result['array_size']:,}",
                f"{result['target']:,}",
                'Yes' if result['found'] else 'No',
                result['position'] if result['position'] != 'Not Found' else 'Not Found',
                result['steps'],
                f"{result['time']:.3f}",
                result['max_steps']
            )
            self.analysis_tree.insert('', tk.END, values=values)
    
    def update_graph(self):
        """Update the performance graph"""
        if not self.analysis_results:
            messagebox.showinfo("No Data", "No analysis data available to plot")
            return

        try:
            self.figure.clear()

            # Extract data
            array_sizes = [r['array_size'] for r in self.analysis_results]
            times = [r['time'] for r in self.analysis_results]

            # Create single plot
            ax = self.figure.add_subplot(1, 1, 1)

            # Plot execution time vs array size
            ax.scatter(array_sizes, times, color='#2E86AB', alpha=0.8, s=80, 
                      edgecolors='white', linewidth=1, zorder=3)
            
            # Add trend line if we have multiple points
            if len(array_sizes) > 1:
                # Sort for trend line
                sorted_indices = np.argsort(array_sizes)
                sorted_sizes = np.array(array_sizes)[sorted_indices]
                sorted_times = np.array(times)[sorted_indices]
                
                ax.plot(sorted_sizes, sorted_times, color='#A23B72', 
                       linestyle='--', alpha=0.7, linewidth=2, zorder=2)
            
            # Customize the plot
            ax.set_xlabel('Array Size (n)', fontsize=12, fontweight='bold')
            ax.set_ylabel('Execution Time (microseconds)', fontsize=12, fontweight='bold')
            ax.set_title('Binary Search: Execution Time vs Array Size', 
                        fontsize=14, fontweight='bold', pad=20)
            ax.grid(True, alpha=0.3, zorder=1)
            
            # Add some styling
            ax.tick_params(axis='both', which='major', labelsize=10)
            
            # Format x-axis for large numbers
            if max(array_sizes) >= 10000:
                ax.ticklabel_format(style='scientific', axis='x', scilimits=(0,0))
            
            # Add data point count
            ax.text(0.02, 0.98, f'Data points: {len(array_sizes)}', 
                   transform=ax.transAxes, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8),
                   fontsize=9)

            # Improve layout
            self.figure.tight_layout()
            
            # Force canvas update
            self.canvas.draw()
            self.canvas.flush_events()
            
            # Update the display
            self.root.update_idletasks()
            
        except Exception as e:
            print(f"Error updating graph: {e}")
            messagebox.showerror("Graph Error", f"Error updating graph: {e}")
    
    def clear_analysis(self):
        """Clear all analysis data"""
        self.analysis_results = []
        self.update_analysis_table()
        self.initialize_empty_graph()
        messagebox.showinfo("Cleared", "Analysis data has been cleared")

def main():
    """Main function to run the application"""
    root = tk.Tk()
    app = SimpleBinarySearchAnalyzer(root)
    root.mainloop()

if __name__ == "__main__":
    main()